# Explore QA RAG Chains with LCEL

## Install OpenAI, and LangChain dependencies

In [1]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install langchain-chroma==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 24.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.42
    Uninstalling langsmith-0.3.42:
      Successfully uninstalled langsmith-0.3.42
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<

## Setup Environment Variables

In [1]:
from google.colab import userdata
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [2]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Vector Databases

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector database takes care of storing embedded data and performing vector search for you.

### Chroma Vector DB

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

### Create a Vector DB and persist on disk

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [3]:
docs = [
 'Quantum mechanics describes the behavior of very small particles.',
 'Photosynthesis is the process by which green plants make food using sunlight.',
 'Artificial Intelligence aims to create machines that can think and learn.',
 'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
 'New Delhi is the capital of India and the seat of all three branches of the Government of India.',
 'Biology is the study of living organisms and their interactions with the environment.',
 'Music therapy can aid in the mental well-being of individuals.',
 'Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.',
 'The Milky Way is just one of billions of galaxies in the universe.',
 'Economic theories help understand the distribution of resources in society.',
 'Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.',
 'Yoga is an ancient practice that involves physical postures and meditation.'
]

In [4]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes 1 min on Colab
chroma_db = Chroma.from_texts(texts=docs, collection_name='db_docs',
                              # need to set the distance function to cosine else it uses euclidean by default
                              # check https://docs.trychroma.com/guides#changing-the-distance-function
                              collection_metadata={"hnsw:space": "cosine"},
                              embedding=openai_embed_model)

## Setup a Vector Database Retriever

Here we use the following retrieval strategy:

- Similarity with Threshold Retrieval


### Similarity with Threshold Retrieval

We use cosine similarity here and retrieve the top 3 similar documents based on the user input query and also introduce a cutoff to not return any documents which are below a certain similarity threshold

In [5]:
similarity_threshold_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                                        search_kwargs={"k": 3,
                                                                       "score_threshold": 0.3})

In [6]:
query = "what is the capital of India?"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.'),
 Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.')]

In [7]:
query = "what is the old capital of India?"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

[Document(metadata={}, page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.'),
 Document(metadata={}, page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(metadata={}, page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.')]

## Build a QA RAG Chain

To build a RAG chain we need a prompt template which instructs the LLM to not answer questions beyond the scope of the retrieved context documents, there are various such prompts out there, we craft one ourselves below

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If no context is present or if you don't know the answer, just say that you don't know.
            Do not make up the answer unless it is there in the provided context.
            Keep the answer concise and to the point with regard to the question.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [9]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

## Legacy LangChain Syntax for QA RAG Chain

Here we show you how you can create your own QA RAG Chain with legacy syntax which works well but not recommended as LangChain has been migrating to LCEL chains.

In [10]:
from langchain.chains import RetrievalQA

In [11]:
qa_rag_chain = RetrievalQA.from_chain_type(llm=chatgpt,
                                           chain_type="stuff",
                                           retriever=similarity_threshold_retriever,
                                           chain_type_kwargs={"prompt": prompt_template})

In [12]:
query = "What is the capital of India?"
qa_rag_chain.invoke(query)

{'query': 'What is the capital of India?',
 'result': 'New Delhi is the capital of India.'}

In [13]:
query = "Tell me about new delhi in detail"
qa_rag_chain.invoke(query)

{'query': 'Tell me about new delhi in detail',
 'result': 'New Delhi is the capital of India and serves as the seat of all three branches of the Government of India.'}

In [14]:
query = "Tell me about the financial capital of India"
qa_rag_chain.invoke(query)

{'query': 'Tell me about the financial capital of India',
 'result': 'Mumbai is the financial capital of India, as well as the most populous city, serving as the financial, commercial, and entertainment capital of South Asia.'}

In [15]:
query = "Who was the winner of the champions league in 2020?"
qa_rag_chain.invoke(query)

{'query': 'Who was the winner of the champions league in 2020?',
 'result': "I don't know."}

## LCEL Syntax for QA RAG Chain - Recommended

Here we will see how to create the RAG chain using LangChain's recommended LCEL

In [16]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_threshold_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
      |
    prompt_template
      |
    chatgpt
)

In [17]:
query = "What is the capital of India?"
result = qa_rag_chain.invoke(query)
print(result.content)

New Delhi is the capital of India.


In [18]:
query = "Tell me about the financial capital of India in detail"
result = qa_rag_chain.invoke(query)
print(result.content)

Mumbai is the financial capital of India, known for being the most populous city in the country. It serves as the financial, commercial, and entertainment hub of South Asia.


In [19]:
query = "What is the fastest animal?"
result = qa_rag_chain.invoke(query)
print(result.content)

I don't know.


In [20]:
query = "How do plants survive?"
result = qa_rag_chain.invoke(query)
print(result.content)

Plants survive by using photosynthesis to make food from sunlight.
